In [1]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot


In [2]:
# default values

modal = 'bold' #structural
TE_min = 0
TE_max = 0.05
TR_min = 0
TR_max = 4
Tesla = 1.5
ModelName = "Prisma_fit"
Manufacturer = "Siemens"

In [3]:
to_filter = ("TE_max", "Tesla", "Manufacturer")
filter_list=[]
filter_dict={'TR_min': "TR >= {}".format(TR_min),
             'TR_max': "TR < {}".format(TR_max),
             'TE_min': "TE >= {}".format(TE_min),
             'TE_max': "TE < {}".format(TE_max)
             }

filter_dict['Tesla'] = "Tesla == 3"
filter_dict['Manufacturer'] = "Manufacturer == 'Siemens'"
filter_dict['ModelName'] = "ModelName == Prisma_fit"

print(to_filter)

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)
    
print(filter_list)
#filter_list= ['TR > 2.2', 'TR < 3']

('TE_max', 'Tesla', 'Manufacturer')
['TE < 0.05', 'Tesla == 3', "Manufacturer == 'Siemens'"]


In [4]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# scan type to query the API for [bold, T1w, T2w]
modality = 'T2w'

# path to input of local data from MRIQC on your own dataset 
#group_file = os.path.join(here,'test_data', 'group_bold.tsv')
group_file = os.path.join('/Users/cmelzer/VOLUMES/tnc_scratch/cmelzer/test/', 'mriqc_taoism', 'group_%s.tsv' %modality)

# remove outliers
outliers = True

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   Tesla, TE, TR
   NOTE: Only working as *and* right now!
"""



# IQM variables to visualize
#need to add separate IQMs for structural and functional
IQM_to_plot = ['fwhm_avg', 'fwhm_x', 'fwhm_y', 'fwhm_z',
               'summary_bg_k', 'summary_bg_mad', 'summary_bg_mean', 'summary_bg_median', 'summary_bg_n', 'summary_bg_p05', 'summary_bg_p95','summary_bg_stdv',
               'efc', 'fber',]
if modal == 'structural':
   IQM_to_plot += ['cjv', 'cnr', 	
                   'icvs_csf', 'icvs_gm', 'icvs_wm',	
                   'inu_med', 'inu_range', 'qi_1', 'qi_2',
                   'rpve_csf', 'rpve_gm', 'rpve_wm', 
                   'snr_csf', 'snr_gm', 'snr_total', 'snr_wm',
                   'snrd_csf', 'snrd_gm',	'snrd_total', 'snrd_wm',		
                   'summary_csf_k', 'summary_csf_mad', 'summary_csf_mean', 'summary_csf_median', 'summary_csf_n', 'summary_csf_p05', 'summary_csf_p95', 'summary_csf_stdv',
                   'summary_gm_k', 'summary_gm_mad', 'summary_gm_mean', 'summary_gm_median', 'summary_gm_n', 'summary_gm_p05', 'summary_gm_p95', 'summary_gm_stdv',
                   'summary_wm_k', 'summary_wm_mad', 'summary_wm_mean', 'summary_wm_median', 'summary_wm_n', 'summary_wm_p05', 'summary_wm_p95', 'summary_wm_stdv',
                   'tpm_overlap_csf', 'tpm_overlap_gm', 'tpm_overlap_wm', 'wm2max']
elif modal == 'bold':
   IQM_to_plot += ['aor', 'aqi', 'dummy_trs', 'dvars_nstd', 'dvars_std', 'dvars_vstd',
                   'fd_mean', 'fd_num', 'fd_perc', 'gcor', 'gsr_x', 'gsr_y', 
                   'snr',  
                   'summary_fg_k', 'summary_fg_mad', 'summary_fg_mean', 'summary_fg_median', 'summary_fg_n', 'summary_fg_p05', 'summary_fg_p95', 'summary_fg_stdv', 
                   'tsnr']
#IQM_to_plot = ['aor','aqi','dummy_trs','dvars_nstd','dvars_std','dvars_vstd',
#                    'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
#                    'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
#                    'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
#                    'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
#                    'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
#                    'summary_fg_p95','summary_fg_stdv','tsnr']

In [5]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

(2, 69)

In [6]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
print(filter_list)
if not filter_list == []:
    (filtered_apidf, query) = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(filtered_apidf.head())
print(query)
#print(list(filtered_apidf))


['TE < 0.05', 'Tesla == 3', "Manufacturer == 'Siemens'"]
(1000, 102)
(205, 102)
['bids_meta_EchoTime<0.05', 'bids_meta_MagneticFieldStrength==3', "bids_meta_Manufacturer=='Siemens'"]


In [7]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



/Users/cmelzer/Develop/mriqception/tools/utils.py:134: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = pd.concat([userdf,filtered_apidf], sort=True).fillna(0)


(207, 103)

### Color coding
Families of IQMs are color-coded. In each plot, the API data is shown in dark gray, and the user's data is shown in the appropriate color. 
- spatial IQMs: gold
- temporal IQMs: orange
- noise IQMs: red
- motion IQMs: green
- artifact IQMs: light blue
- descriptive IQMs: dark blue
- other IQMs: purple

In [8]:
# Visualization # 

# Create a violin plot of the MRIQC metrics, comparing user level-data to API data 
# Also will print out a short description of the measure and what the appropriate range/values should be 
# Input the merged dataframe including the API and user data
# Optional arguments include a list of specific variables that the user wants displayed 
# If no optional arguments included, will plot all measures 

v = make_vio_plot(vis_ready_df,IQM_to_plot,"",outliers=outliers, comment=query)

widgets.VBox([v[0],v[1]])


Loading in dataframe...
Loading variables: ['fwhm_avg', 'fwhm_x', 'fwhm_y', 'fwhm_z', 'summary_bg_k', 'summary_bg_mad', 'summary_bg_mean', 'summary_bg_median', 'summary_bg_n', 'summary_bg_p05', 'summary_bg_p95', 'summary_bg_stdv', 'efc', 'fber', 'aor', 'aqi', 'dummy_trs', 'dvars_nstd', 'dvars_std', 'dvars_vstd', 'fd_mean', 'fd_num', 'fd_perc', 'gcor', 'gsr_x', 'gsr_y', 'snr', 'summary_fg_k', 'summary_fg_mad', 'summary_fg_mean', 'summary_fg_median', 'summary_fg_n', 'summary_fg_p05', 'summary_fg_p95', 'summary_fg_stdv', 'tsnr']
Loading in data descriptors...


/Users/cmelzer/Develop/mriqception/venv/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



/Users/cmelzer/Develop/mriqception/venv/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

...

/Users/cmelzer/Develop/mriqception/venv/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

IndexError: index 0 is out of bounds for axis 0 with size 0

/Users/cmelzer/Develop/mriqception/venv/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/Users/cmelzer/Develop/mriqception/venv/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

